In [1]:
from keras.models import Sequential,Model
from keras import layers
from keras import Input

model=Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

input_tensor=Input(shape=(64,))
x=layers.Dense(32,activation='relu')(input_tensor)
x=layers.Dense(32,activation='relu')(x)
output_tensor=layers.Dense(10,activation='softmax')(x)

model=Model(input_tensor,output_tensor)

model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy')

import numpy as np
X_train=np.random.random((1000,64))
y_train=np.random.random((1000,10))

model.fit(X_train,y_train,epochs=10,batch_size=128)
score=model.evaluate(X_train,y_train)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1000/1000 [==============================] - 7s 7ms/step - loss: 11.4594
Epoch 2/10
1000/1000 [==============================] - 0s 22us/step - loss: 11.4219
Epoch 3/10
1000/1000 [==============================] - 0s 14us/step - loss: 11.4118
Epoch 4/10
1000/10

In [6]:
from keras.models import Sequential,Model
from keras import layers
from keras import Input

text_vocabulary_size=10000
question_vocabulary_size=10000
answer_vocabulary_size=500

text_input=Input(shape=(None,),dtype='int32',name='text')

embedded_text=layers.Embedding(text_vocabulary_size,64)(text_input)

encoded_text=layers.LSTM(32)(embedded_text)

question_input=Input(shape=(None,),dtype='int32',name='question')
embedded_question=layers.Embedding(question_vocabulary_size,32)(question_input)
encoded_question=layers.LSTM(16)(embedded_question)

concatenated=layers.concatenate([encoded_text,encoded_question],axis=-1)

answer=layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)

model=Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

In [8]:
import numpy as np
num_samples=1000
max_length=100

text=np.random.randint(1,text_vocabulary_size,size=(num_samples,max_length))
question=np.random.randint(1,question_vocabulary_size,size=(num_samples,max_length))

answers=np.zeros(shape=(num_samples,answer_vocabulary_size))
indices=np.random.randint(0,answer_vocabulary_size,size=num_samples)
for i,x in enumerate(answers):
    x[indices[i]]=1
    
model.fit([text,question],answers,epochs=10,batch_size=128)

model.fit({'text':text,'question':question},answers,epochs=10,batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.2143 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1975 - acc: 0.0350
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1419 - acc: 0.0090
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0586 - acc: 0.0060
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0049 - acc: 0.0070
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9577 - acc: 0.0070
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9000 - acc: 0.0110
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8270 - acc: 0.0100
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.7469 - acc: 0.0180
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6497 - acc: 0.0290
Epoch 1/10
1000/1000 [=======

In [16]:
from keras.models import Sequential,Model
from keras import layers
from keras import Input

vocabulary_size=50000
num_income_groups=10

posts_input=Input(shape=(None,),dtype='int32',name='posts')
embedded_posts=layers.Embedding(256,vocabulary_size)(posts_input)
x=layers.Conv1D(128,5,activation='relu')(embedded_posts)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.MaxPooling1D(5)(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.Conv1D(256,5,activation='relu')(x)
x=layers.GlobalMaxPooling1D()(x)
x=layers.Dense(128,activation='relu')(x)

age_prediction=layers.Dense(1,name='age')(x)
income_prediction=layers.Dense(num_income_groups,activation='softmax',name='income')(x)
gender_prediction=layers.Dense(1,activation='sigmoid',name='gender')(x)

model=Model(posts_input,[age_prediction,income_prediction,gender_prediction])

model.compile(optimizer='rmsprop',
              loss=['mse',
                   'binary_crossentropy',
                   'categorical_crossentropy'],
              loss_weights=[0.25,1.,10.])

model.compile(optimizer='rmsprop',
              loss={'age':'mse',
                    'income':'categorical_crossentropy',
                    'gender':'binary_crossentropy'},
              loss_weights={'age':0.25,'income':1.,'gender':10.})

model.summary()

model.fit(posts,[age_targets,income_targets,gender_targets])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, None, 128)    32000128    embedding_11[0][0]               
__________________________________________________________________________________________________
max_pooling1d_13 (MaxPooling1D) (None, None, 128)    0           conv1d_31[0][0]                  
__________________________________________________________________________________________________
conv1d_32 

NameError: name 'posts' is not defined

In [13]:
from keras import layers
from keras import Input

x=Input(shape=(299,3))
branch_a=layers.Conv2D(128,1,activation='relu',strides=2)(x)

branch_b=layers.Conv2D(128,1,activation='relu',strides=2)(x)
branch_b=layers.Conv2D(128,3,activation='relu')(branch_b)

branch_c=layers.AveragePooling2D(3,strides=2)(x)
branch_c=layers.Conv2D(128,3,activation='relu')(branch_c)

branch_d=layers.Conv2D(128,1,activation='relu')(x)
branch_d=layers.Conv2D(128,3,activation='relu')(branch_d)
branch_d=layers.Conv2D(128,3,activation='relu',strides=2)(branch_d)

output=layers.concatenate([branch_a,branch_b,branch_c,branch_d],axis=-1)



ValueError: Input 0 is incompatible with layer conv2d_103: expected ndim=4, found ndim=3

In [12]:
from keras.applications.inception_v3 import InceptionV3
model=InceptionV3()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 149, 149, 32) 864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_9[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_10 

In [14]:
from keras import layers
x=Input(shape=(None,))
y=layers.Conv2D(128,3,activation='relu',padding='same')(x)
y=layers.Conv2D(128,3,activation='relu',padding='same')(y)
y=layers.Conv2D(128,3,activation='relu',padding='same')(y)
y=layers.add([y,x])

y=layers.Conv2D(128,3,activation='relu',padding='same')(x)
y=layers.Conv2D(128,3,activation='relu',padding='same')(y)
y=layers.MaxPooling2D(2,1,strides=2)(y)

residual=layers.Conv2D(128,1,strides=2,padding="same")(y)
y=layers.add([residual,x])



ValueError: Input 0 is incompatible with layer conv2d_104: expected ndim=4, found ndim=2

In [17]:
from keras.models import Model

lstm=layers.LSTM(32)

left_input=Input(shape=(None,128))
left_output=lstm(left_input)

right_input=Input(shape=(None,128))
right_output=lstm(right_input)

merged=layers.concatenate([left_output,right_output],axis=-1)
predictions=layers.Dense(1,activation="sigmoid")(merged)

model=Model([left_input,right_input],predictions)

model.summary()
#model.fit([left_data,right_data],targets)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 32)           20608       input_14[0][0]                   
                                                                 input_15[0][0]                   
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 64)           0           lstm_3[0][0]                     
          

In [23]:
from keras import layers
from keras import applications
from keras import Input

xception_base=applications.Xception(weights=None,include_top=False)

left_input=Input(shape=(250,250,3))
right_input=Input(shape=(250,250,3))

left_features=xception_base(left_input)
right_input=xception_base(right_input)

merged_features=layers.concatenate([left_features,right_input],axis=-1)


/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/keras/engine/network.py:180: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer xception.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: xception_7/block14_sepconv2_act/Relu:0
  str(x.name))


AssertionError: 

In [25]:
merged_features.shape

TensorShape([Dimension(None), Dimension(8), Dimension(8), Dimension(4096)])

In [ ]:
import keras

callbacks_list=[
        keras.callbacks.EarlyStopping(
            monitor='val_acc',
            patience=1),
        keras.callbacks.ModelCheckpoint(
            filepath='my_model.h5',
            monitor='val_loss',
            save_best_only=True,)]

model.fit(x,y,epochs=16,batch_size=128,callbacks=callbacks_list,validation_data=(x_val,y_val))

In [ ]:
import keras
callbacks_list=[
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=10)]

model.fit(x,y,epochs=16,batch_size=128,callbacks=callbacks_list,validation_data=(x_val,y_val))

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features=2000
max_len=50

(X_train_origin,y_train),(X_test_origin,y_test)=imdb.load_data(num_words=max_features)
X_train=sequence.pad_sequences(X_train_origin,maxlen=max_len,truncating='post')
X_test=sequence.pad_sequences(X_test_origin,maxlen=max_len,truncating='post')

model=keras.models.Sequential()
model.add(layers.Embedding(max_features,128,input_length=max_len,name='embed'))
model.add(layers.Conv1D(32,7,activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32,7,activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 50, 128)           256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 44, 32)            28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 32)             7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [2]:
callbacks=[
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',
        histogram_freq=1,
        embeddings_freq=1,
        embeddings_data=X_train[:50])
]

history=model.fit(X_train,y_train,epochs=20,batch_size=128,callbacks=callbacks,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 2s 75us/step - loss: 0.6688 - acc: 0.6221 - val_loss: 0.5506 - val_acc: 0.7220
Epoch 2/20
20000/20000 [==============================] - 0s 22us/step - loss: 0.5278 - acc: 0.7612 - val_loss: 0.5854 - val_acc: 0.7298
Epoch 3/20
20000/20000 [==============================] - 0s 20us/step - loss: 0.4495 - acc: 0.8045 - val_loss: 0.6660 - val_acc: 0.7192
Epoch 4/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.3761 - acc: 0.8267 - val_loss: 0.7289 - val_acc: 0.7110
Epoch 5/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.2607 - acc: 0.8350 - val_loss: 1.1798 - val_acc: 0.6368
Epoch 6/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.1658 - acc: 0.7700 - val_loss: 1.4232 - val_acc: 0.6020
Epoch 7/20
20000/20000 [==============================] - 0s 18us/step - loss: 0.0966 - acc: 0.6622 - val_loss: 1.8301 - val_acc:

In [3]:
from keras.utils import plot_model
plot_model(model,show_shapes=True,to_file='model.png')

In [9]:
from keras.models import Sequential,Model
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import os
dataset_dir="/home/nagae/Desktop/research/imai/experiment_code/cifar/cifar10"
train_datagen=ImageDataGenerator()
test_datagen=ImageDataGenerator()
image_shape=(250,250,3)

train_generator = train_datagen.flow_from_directory(
        os.path.join(dataset_dir, 'train'), 
        target_size=image_shape[:2], 
        batch_size=64, 
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        os.path.join(dataset_dir, 'test'), 
        target_size=image_shape[:2], 
        batch_size=64, 
        class_mode='categorical')





Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [12]:
from keras.models import Sequential
from keras import layers

model=Sequential()
model.add(layers.SeparableConv2D(32,3,activation='relu',input_shape=(250,250,3)))
model.add(layers.SeparableConv2D(64,3,activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(32,3,activation='relu',input_shape=(250,250,3)))
model.add(layers.SeparableConv2D(128,3,activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(32,3,activation='relu',input_shape=(250,250,3)))
model.add(layers.SeparableConv2D(128,3,activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy')

history = model.fit_generator(
        train_generator,  
        steps_per_epoch=50000 // 64, 
        epochs=10, 
        validation_data=test_generator, 
        validation_steps=50000 // 64,  
        verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_7 (Separabl (None, 248, 248, 32)      155       
_________________________________________________________________
separable_conv2d_8 (Separabl (None, 246, 246, 64)      2400      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 123, 123, 64)      0         
_________________________________________________________________
separable_conv2d_9 (Separabl (None, 121, 121, 32)      2656      
_________________________________________________________________
separable_conv2d_10 (Separab (None, 119, 119, 128)     4512      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 59, 59, 128)       0         
_________________________________________________________________
separable_conv2d_11 (Separab (None, 57, 57, 32)        5280      
__________

KeyboardInterrupt: 

In [2]:
from keras.applications import Xception
model=Xception()
model.summary()

from keras.utils import plot_model
plot_model(model,show_shapes=True,to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [8]:
import cv2
img=cv2.imread('model.png')
cv2.imshow("color",img)

error: OpenCV(3.4.2) /tmp/build/80754af9/opencv-suite_1535558553474/work/modules/highgui/src/window.cpp:632: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
